In [2]:
!pip install opencv-python
!pip install scikit-fuzzy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 9.5 MB/s eta 0:00:00


In [5]:
import cv2
import numpy as np
import skfuzzy as fuzz
import skfuzzy.membership as mf

def preprocess_image(image_path):
    """Loads an image, converts it to grayscale, and applies thresholding."""
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"Image at path {image_path} could not be loaded.")

    _, binary_img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)
    return binary_img

def extract_features(image):
    """Extracts simple geometric features: aspect ratio, area, and contour complexity."""
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None

    cnt = max(contours, key=cv2.contourArea)  # Select the largest contour
    x, y, w, h = cv2.boundingRect(cnt)
    aspect_ratio = w / float(h) if h != 0 else 0  # Avoid division by zero
    area = cv2.contourArea(cnt)
    perimeter = cv2.arcLength(cnt, True)
    complexity = perimeter ** 2 / area if area > 0 else 0

    return aspect_ratio, area, complexity

def define_fuzzy_sets():
    """Defines fuzzy membership functions for shape features."""
    x_ar = np.linspace(0, 2, 100)
    x_area = np.linspace(0, 5000, 100)
    x_complexity = np.linspace(0, 500, 100)

    ar_low = mf.trimf(x_ar, [0, 0.5, 1])
    ar_medium = mf.trimf(x_ar, [0.5, 1, 1.5])
    ar_high = mf.trimf(x_ar, [1, 1.5, 2])

    area_small = mf.trimf(x_area, [0, 500, 1500])
    area_medium = mf.trimf(x_area, [1000, 2500, 4000])
    area_large = mf.trimf(x_area, [3000, 4000, 5000])

    comp_simple = mf.trimf(x_complexity, [0, 50, 150])
    comp_moderate = mf.trimf(x_complexity, [100, 200, 300])
    comp_complex = mf.trimf(x_complexity, [250, 400, 500])

    fuzzy_sets = {
        'aspect_ratio': (x_ar, [ar_low, ar_medium, ar_high]),
        'area': (x_area, [area_small, area_medium, area_large]),
        'complexity': (x_complexity, [comp_simple, comp_moderate, comp_complex])
    }

    return fuzzy_sets

def match_character(features, fuzzy_sets):
    """Matches a character based on fuzzy logic."""
    x_ar, ar_sets = fuzzy_sets['aspect_ratio']
    x_area, area_sets = fuzzy_sets['area']
    x_complexity, comp_sets = fuzzy_sets['complexity']

    # Fuzzy membership for each feature
    ar_membership = [fuzz.interp_membership(x_ar, ar, features[0]) for ar in ar_sets]
    area_membership = [fuzz.interp_membership(x_area, area, features[1]) for area in area_sets]
    complexity_membership = [fuzz.interp_membership(x_complexity, comp, features[2]) for comp in comp_sets]

    # Return the fuzzy membership values
    return {
        'aspect_ratio': ar_membership,
        'area': area_membership,
        'complexity': complexity_membership
    }

def similarity_score(match_result):
    """Calculates a similarity score based on fuzzy membership values."""
    # The similarity score can be calculated by averaging the membership degrees
    aspect_ratio_score = max(match_result['aspect_ratio'])
    area_score = max(match_result['area'])
    complexity_score = max(match_result['complexity'])

    # Aggregate the scores to get a final similarity score
    final_score = (aspect_ratio_score + area_score + complexity_score) / 3
    return final_score

# Example usage
def main():
    image_path = '/content/Handwriting-test-dataset-for-OCR-operation.png'  # Replace with the actual image path
    try:
        # Preprocess the image and extract features
        binary_img = preprocess_image(image_path)
        features = extract_features(binary_img)

        if features:
            fuzzy_sets = define_fuzzy_sets()
            match_result = match_character(features, fuzzy_sets)
            print("Fuzzy membership values:", match_result)

            # Calculate the similarity score
            score = similarity_score(match_result)
            print(f"Similarity Score: {score}")

            # Based on the score, you can classify or match shapes to predefined categories
            if score > 0.7:
                print("The character is likely a match!")
            else:
                print("The character does not match.")
        else:
            print("No character detected.")

    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()

Fuzzy membership values: {'aspect_ratio': [0.6666666666666666, 0.0, 0.0], 'area': [0.009000000000000001, 0.0, 0.0], 'complexity': [0.6917652825134307, 0.0, 0.0]}
Similarity Score: 0.45581064972669916
The character does not match.


In [6]:
import numpy as np

# ======================== FUZZY MEMBERSHIP FUNCTIONS ========================
def curvature_fuzzy_set(curvature):
    flat = max(0, min(1, (10 - curvature)/10))
    moderately_curved = max(0, min(1, (curvature-10)/10, (30-curvature)/10))
    highly_curved = max(0, min(1, (curvature-30)/10))
    return flat, moderately_curved, highly_curved

def stroke_direction_fuzzy_set(angle):
    horizontal = max(0, min(1, (45 - abs(angle))/45)
    vertical = max(0, min(1, (45 - abs(angle-90))/45)
    diagonal = max(0, min(1, (45 - abs(angle-45))/45)
    return horizontal, vertical, diagonal

def loop_fuzzy_set(loop_type):
    no_loop = 1 if loop_type == "none" else 0
    partial_loop = 1 if loop_type == "partial" else 0
    full_loop = 1 if loop_type == "full" else 0
    return no_loop, partial_loop, full_loop

def symmetry_fuzzy_set(symmetry):
    symmetric = 1 if symmetry == "symmetric" else 0
    asymmetric = 1 if symmetry == "asymmetric" else 0
    return symmetric, asymmetric

def line_segment_fuzzy_set(num_segments):
    few = max(0, min(1, (3 - num_segments)/3))
    moderate = max(0, min(1, (num_segments-3)/2, (6-num_segments)/3))
    many = max(0, min(1, (num_segments-6)/4))
    return few, moderate, many

# ======================== FUZZY RULES FOR LETTER DETECTION ========================
def detect_letter(curvature, loop, symmetry, segments, direction):
    """Detects letters using fuzzy rules (O, L, S detection)"""
    # Rule for "O" (high curvature, full loop, symmetry)
    if curvature[2] > 0.3 and loop[2] > 0.3 and symmetry[0] > 0.3:
        return "O"

    # Rule for "L" (flat curvature, no loop, vertical direction)
    elif curvature[0] > 0.3 and loop[0] > 0.3 and direction[1] > 0.7:
        return "L"

    # Rule for "S" (moderate curvature, partial loop, diagonal direction)
    elif curvature[1] > 0.3 and loop[1] > 0.3 and direction[2] > 0.5:
        return "S"

    # Add more letter rules here...

    return "Unknown"

# ======================== EXAMPLE USAGE ========================
if __name__ == "__main__":
    # Sample features for letter "O"
    curvature = curvature_fuzzy_set(35)  # High curvature
    loop = loop_fuzzy_set("full")        # Full loop
    symmetry = symmetry_fuzzy_set("symmetric")
    direction = stroke_direction_fuzzy_set(10)  # Nearly horizontal

    detected_char = detect_letter(curvature, loop, symmetry, None, direction)
    print(f"Detected letter: {detected_char}")  # Output: "O"

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-6-b7d37a147766>, line 11)